## Huffman

O código abaixo implementa uma árvore representando um **código de Huffman**, gerado a partir de um conjunto de caracteres e suas frequências.

O código abaixo implementa uma lista de prioridades usando um Heap Mínimo. Cada elemento da lista de prioridades será um nó com uma frequência. A lista será ordenada usando a propriedade de **heap mínimo** pela frequência.


In [1]:
#include <iostream>
#include <stdlib.h> 
using namespace std;

#define ALFABETO 6;
#define BITS_CODIGO 6;



Estrutura usada para armazenar, ao mesmo tempo, uma lista de prioridades e uma árvore de Huffman. 

In [2]:
struct tNo {
    char c;
    int frequencia; 
    tNo *esq, *dir, *pai;
};

Dado um HEAP e o índice de um elemento, esta função retorna o **índice** do elemento pai.

In [3]:
int pai(int i){
    if (i > 0)
        return (i-1)/2;
    else
        return -1;
}

Esta função retorna o **índice** do elemento da esquerda.

In [4]:
int esq(int i){
    return 2*i+1;
}

Esta função retorna o **índice** do elemento da direita.

In [5]:
int dir(int i){
    return 2*i+2;
}

Retorna o tamanho do vetor do HEAP. Nesta implementação o tamanho é fixo.

In [6]:
int tam(){
    return ALFABETO;
}

Imprime o heap mínimo em pré-ordem.

In [7]:
void preordemFila (tNo *A[], int indice, int tamHeap){
    if (indice < tamHeap){
        cout << "(" << A[indice]->c << ":" << A[indice]->frequencia << ".";
        preordemFila(A,esq(indice),tamHeap);        
        preordemFila(A,dir(indice),tamHeap);
        cout << ")";
    }
}

Imprime uma árvore de Huffman em pré-ordem.

In [8]:
void preordemArvoreHuffman (tNo *no){
    if (no != NULL){
        cout << "(" << no->c << ":" << no->frequencia << ".";
        preordemArvoreHuffman(no->esq);       
        preordemArvoreHuffman(no->dir);
        cout << ")";
    }
}

Troca dois elementos da lista, de acordo com os índices passados como parâmetro.

In [9]:
void troca(tNo *A[], int i, int j){    
    tNo *aux = A[i];
    A[i] = A[j];
    A[j] = aux;
}

Esta função ajusta para que fique de acordo com a propriedade *heap mínimo*. A função assume que a subárvore já está seguindo a propriedade heap mínimo.

In [10]:
void min_heapify(tNo *A[], int i, int tamHeap){
    int e = esq(i), d = dir(i);
    int menor;
    if (e < tamHeap && A[e]->frequencia < A[i]->frequencia)
        menor = e;
    else
        menor = i;
    if (d < tamHeap && A[d]->frequencia < A[menor]->frequencia)
        menor = d;
    if (menor  != i){
        troca(A, i, menor);
        min_heapify(A, menor, tamHeap);
    }        
}

Constroi um heap mínimo a partir de um vetor de nós desordenados (_frequencia_ como chave).

In [11]:
void constroi_min_heap(tNo *A[], int tamHeap){
    int indice = (tamHeap-1)/2;
    for (int i=indice; i>=0; i--){
        min_heapify(A,i, tamHeap);
    }
}

Retorna o elemento mínimo do heap, i.e., o primeiro elemento.

In [12]:
tNo *minimo(tNo *A[]){
    return A[0];
}

Extrai o primeiro elemento (retira do heap), atribui o valor do último elemento para o primeiro e executa o min heapify.

In [13]:
tNo *extrai_minimo(tNo *A[], int *tamHeap){
    tNo *min = NULL;
    if ((*tamHeap) > 0) {
        min = A[0];
        troca(A,0,(*tamHeap)-1);
        (*tamHeap) --;
        min_heapify(A,0,(*tamHeap));
    }
    return min;
}

Posiciona um nó corretamente no heap, de acordo com a frequência.

In [14]:
void diminui_chave(tNo *A[], int i){
        while (i >= 0 && pai(i) >= 0 && A[pai(i)]->frequencia > A[i]->frequencia){
            troca(A, i, pai(i));
            i = pai(i);
        }
}

Cria um nó para ser incluído na lista de prioridades, e posteriormente na árvore de Huffman.

In [15]:
tNo *criaNoHeap(int frequencia, char c){
    tNo *noHeap = (tNo *)malloc(sizeof(tNo));
    noHeap->frequencia = frequencia;
    noHeap->c = c;
    noHeap->esq = NULL;
    noHeap->dir = NULL;
    noHeap->pai = NULL;
    return noHeap;
}

Inclui nova chave, até o tamanho máximo da lista.

In [16]:
void inclui (tNo *A[], tNo *novo, int *tamHeap){
    int tamMax = ALFABETO;
    if ((*tamHeap) < tamMax){
        (*tamHeap)++;
        A[(*tamHeap)-1] = novo;
        diminui_chave(A, (*tamHeap)-1);
    }
    else
        cout << "\nNão pode incluir nova chave, tamanho do vetor não suportado\n";
}

Função que cria uma árvore representando o código de Huffman. 

In [17]:
void huffman (tNo *A[], int *tamHeap){    
    tNo *novo = NULL;
    while ((*tamHeap) > 1) {
        novo = criaNoHeap(0,'_');
        novo->esq = extrai_minimo(A,tamHeap);
        novo->esq->pai = novo;
        novo->dir = extrai_minimo(A,tamHeap);
        novo->dir->pai = novo;
        novo->frequencia = novo->esq->frequencia + novo->dir->frequencia;
        inclui(A, novo, tamHeap);
    }
}

Função usada par~a recuperar o índice da letra armazenada na tabela com o código Huffman.

O índice é dado pelo MOD (módulo da divisão inteira) da letra pelo tamanho máximo da tabela. Por isso, o alfabeto deverá conter letras, de A até o tamanho da tabela. Esta restrição pode ser modificada, desde que a estrutura da tabela também seja alterada.

In [18]:
int get_indice_tabela(char c){
    int maxBits = BITS_CODIGO;
    return (toupper(c) - 65) % maxBits;
}

Constroi uma tabela de acordo com o código Huffman gerado. Percorre os nós até os folhas, e volta invertido até o pai para armazenar o código em uma tabela.
O índice da tabela é dado pelo MOD 6 da letra.
 

In [19]:
void constroiTabelaHuffman (char t[][6], tNo *no, int nivel){
    int i;
    int indice;
    if (no != NULL){
        constroiTabelaHuffman(t, no->esq, nivel + 1);       
        constroiTabelaHuffman(t, no->dir, nivel + 1);
        if (no->esq == NULL && no->dir == NULL) {
            indice = get_indice_tabela(no->c);
            i = 0;
            while (no != NULL && no->pai != NULL ) {
                if (no->pai->esq == no)
                    t[indice][nivel-i-1] = '0';
                else
                    t[indice][nivel-i-1] = '1';
                no = no->pai;
                i++;
            }
            t[indice][i] = '\0';
        }
    }
}

Função que codifica um conjunto de caracteres de acordo com a tabela com o código de Huffman gerada.

In [20]:
void codifica(const char *str, char tabela[][6]){
    for (int i=0; str[i] != '\0'; i++){
        cout << tabela[get_indice_tabela(str[i])] ;
    }
}

Função que decodifica um conjunto de caracteres de acordo com a árvore de Huffman.

In [21]:
void decodifica(const char *str, tNo *no){
    tNo *raiz = no;
    for (int i=0; str[i] != '\0'; i++){
        if (str[i] == '0')
            no = no->esq;
        else 
            no = no->dir;
        if (no->dir == NULL && no->esq == NULL){
            cout << no->c;
            no = raiz;
        }
    }
}

In [22]:
void iniciaprograma(){
    
    int bits = BITS_CODIGO;
    tNo *A[12] = {criaNoHeap(45,'a'),criaNoHeap(13,'b'),criaNoHeap(12,'c'),
                  criaNoHeap(16,'d'),criaNoHeap(9,'e'),criaNoHeap(5,'f'),
               NULL,NULL,NULL,NULL,NULL,NULL};
    int tamHeapA = ALFABETO;
    char tabelaHuffman [tamHeapA][6];
    
    constroi_min_heap(A, tamHeapA);
    cout << "fila de prioridade: \n";
    preordemFila(A, 0, tamHeapA);
    
    huffman (A, &tamHeapA);
    cout << "\nfila após execução do algoritmo de Huffman";
    preordemFila(A,0,tamHeapA);    
    cout << "\nárvore de Huffman" << endl;
    preordemArvoreHuffman(minimo(A));
    cout << endl;
                        
    constroiTabelaHuffman(tabelaHuffman, minimo(A),0);
    
    cout << "\ntabela Huffman";
     for (int i =0; i < 6; i ++){
         cout << "\n"<< (char)(i+97) << ":" ;
         cout << tabelaHuffman[i] ;
     }
    
    cout << "\n\n" << "addffbadaae\0" << " codificado\n";
    codifica("addffbadaae\0",tabelaHuffman);
    
    cout << "\n" << "ffff\0" << " codificado\n";
    codifica("ffff\0",tabelaHuffman);
    
    cout << "\n\n" << "1100110011001100\0" << " de-codificado\n";
    decodifica("1100110011001100\0",minimo(A));    
    cout << "\n\n" << "0111111110011001010111001101\0" << " de-codificado\n";
    decodifica("0111111110011001010111001101\0",minimo(A));      

}

In [23]:
iniciaprograma();

fila de prioridade: 
(f:5.(e:9.(d:16.)(b:13.))(c:12.(a:45.)))
fila após execução do algoritmo de Huffman(_:100.)
árvore de Huffman
(_:100.(a:45.)(_:55.(_:25.(c:12.)(b:13.))(_:30.(_:14.(f:5.)(e:9.))(d:16.))))

tabela Huffman
a:0
b:101
c:100
d:111
e:1101
f:1100

addffbadaae codificado
0111111110011001010111001101
ffff codificado
1100110011001100

1100110011001100 de-codificado
ffff

0111111110011001010111001101 de-codificado
addffbadaae